## 1.9 Model IO

# Model IO Exercise Solution

The purpose of this exercise is to test your understanding of building out Model IO systems. You will also hopefully notice the need to chain responses together, which we will cover later in this course!

Watch the video for a full overview on minimum outputs this class should be capable of, but feel free to expand on this project, or to just treat it as a code-along!

## History Quiz

Our main goal is to use LangChain and Python to create a very simple class with a few methods for:
* Writing a historical question that has a date as the correct answer
* Getting the correct answer from LLM
* Getting a Human user's best guess at at correct answer
* Checking/reporting the difference between the correct answer and the user answer

In [1]:
!pip install -U langchain-community

%pip install --upgrade --quiet  langchain-huggingface text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2 bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB

In [4]:
!pip install langchain_huggingface

In [1]:
from google.colab import userdata
HUGGINGFACEHUB_API_TOKEN= userdata.get('HUGGING_FACE_API_KEY')

In [2]:
from datetime import datetime
import transformers, torch

from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)

from langchain.output_parsers import DatetimeOutputParser
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from huggingface_hub import login

In [3]:
login(token = HUGGINGFACEHUB_API_TOKEN)

In [4]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = transformers.pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_new_tokens=2048,
    do_sample=False,
    repetition_penalty=1.03
)
pipe.tokenizer.pad_token = pipe.tokenizer.eos_token  # Set pad_token to eos_token

model_kwargs = {'temperature':0.3}
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = model_kwargs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [5]:
class HistoryQuiz():

    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:

            "On what date did World War 2 end?"

        '''
        # PART ONE: SYSTEM
        system_template="You write single quiz questions about {topic}. You only return the quiz question."
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        # PART TWO: HUMAN REQUEST
        human_template="{question_request}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        # PART THREE: COMPILE TO CHAT
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
        # PART FOUR: INSERT VARIABLES
        request = chat_prompt.format_prompt(topic=topic,question_request="Give me a quiz question where the correct answer is a specific date.").to_messages()
        # PART FIVE: CHAT REQUEST
        chat = ChatHuggingFace(llm=llm)
        result = chat(request)

        return result.content

    def get_AI_answer(self,question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!

        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''
        # Datetime Parser
        output_parser = DatetimeOutputParser()

        # SYSTEM Template
        system_template = "You answer quiz questions with just a date."
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

        # HUMAN Template
        human_template = """Answer the user's question:

        {question}

        {format_instructions}"""
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

        # Compile ChatTemplate
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

        # Insert question and format instructions

        request = chat_prompt.format_prompt(question=question,
                                            format_instructions=output_parser.get_format_instructions()).to_messages()

        # Chat Bot
        chat = ChatHuggingFace(llm=llm)
        result = chat(request)
        # Format Request to datetime
        correct_datetime = output_parser.parse(result.content)
        return correct_datetime

    def get_user_answer(self,question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        print(question)

        # Get the year, month, and day from the user
        year = int(input("Enter the year: "))
        month = int(input("Enter the month (1-12): "))
        day = int(input("Enter the day (1-31): "))

        # Create a datetime object
        user_datetime = datetime(year, month, day)

        return user_datetime


    def check_user_answer(self,user_answer,ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''

        # Calculate the difference between the dates
        difference = user_answer - ai_answer

        # Format the difference into a string
        formatted_difference = str(difference)

        # Return the string reporting the difference
        print("The difference between the dates is:", formatted_difference)



In [6]:
quiz_bot = HistoryQuiz()

In [7]:
question = quiz_bot.create_history_question(topic='World War 2')

<ipython-input-5-d507f8a14fae>:23: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chat(request)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [14]:
question = quiz_bot.create_history_question(topic="World War 2")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [15]:
question

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 10 Dec 2024\n\nYou write single quiz questions about World War 2. You only return the quiz question.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGive me a quiz question where the correct answer is a specific date.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nWhat was the date of the first V-2 rocket launch?'

In [ ]:
ai_answer = quiz_bot.get_AI_answer("World War 2")

# Day After Pearl Harbor
ai_answer

In [11]:
user_answer = quiz_bot.get_user_answer(question)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 10 Dec 2024

You write single quiz questions about World War 2. You only return the quiz question.<|eot_id|><|start_header_id|>user<|end_header_id|>

Give me a quiz question where the correct answer is a specific date.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

What was the date of the first V-2 rocket launch?
Enter the year: 1941
Enter the month (1-12): 12
Enter the day (1-31): 1


In [12]:
user_answer

datetime.datetime(1941, 12, 1, 0, 0)

In [ ]:
quiz_bot.check_user_answer(user_answer,ai_answer)